# Where Should a Data-Science Job Candidate  Move in the Bay Area?

## 1. Problem Description:

There are currently many data science jobs in the Bay Area, which attract many job seekers in this field to the region. However, some of these job seekers first move to the area for traning purposes and they want to decide on a long-term rental place before they know the location of their future job. How can such a person decide which city to move considering the abundance of data science job openings in that city and other criteria affecting their quality of life?

Suppose that your friend, who is looking for a full-time data science job near San Francisco, Bay Area, approaches you with the following problem: She needs to move to the Bay Area before finding a job because she plans to attend a 7-week long data science camp in Silicon Valley before applying for jobs. The long-term rentals are cheaper and she does not want to move again within the Bay Area shortly after finding a job. Therefore, she wants to move to a city which has a high chance of being close to her future job and provides a good quality of life in general. Her criteria for the ideal city to live in are:

1- The city should include companies with job openings for data scientists. The more job openings, the better.

2- The city ideally should have a low crime rate.

3- Given that she is Turkish, she would like the Turkish restaurants nearby to have a high average rating. 

She wants you to create a ranking of the cities best-suited to her criteria above.

## 2. Data Description and Cleaning:

We will describe and clean the data, needed to rank the cities according to three criteria above:

### 2.1. Job Openings for Data Scientists near San Francisco, Bay Area: 

[Indeed.com](https://www.indeed.com) provides full-time job openings for data scientists within 50 mile radius of San Francisco Bay Area, CA. We can use the url for this search, to parse the number of jobs in each city from the web page.

First, we import the libraries necessary for the analysis.

In [23]:
import requests # library for making HTTP requests in Python
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from bs4 import BeautifulSoup # library for pulling data out of HTML and XML files

We can use the url below to parse through Beautiful Soup the html data on [Indeed.com](https://www.indeed.com) looking for full-time job openings with the key word "data scientist" within 50 mile radius of San Francisco Bay Area, CA.

In [24]:
url='https://www.indeed.com/jobs?q=data+scientist&jt=fulltime&l=San+Francisco+Bay+Area%2C+CA&radius=50'
index = requests.get(url).text #Request the text data from Indeed.com
soup = BeautifulSoup(index, 'html.parser') 

We then scrape the data on the side bar of the web page which includes the number of data science jobs in cities within 50 miles from San Francisco Bay Area. Finally, we put the data into a dataframe and split the city and the number of data science job openings into two columns.

In [25]:
city_job = soup.find_all(id="LOCATION_rbo")

df=[cj.get_text(strip=True) for cj in city_job]

df=pd.DataFrame(df)

df=df[0].str.split(")", expand=True).T 
df=df[0].str.split("(", expand=True)
df[0]=df[0].str.split(",", expand=True)
df.columns=['City','Job Openings']
df

,City,Job Openings
0,San Francisco,1439
1,Sunnyvale,529
2,San Jose,258
3,Palo Alto,247
4,Menlo Park,241
5,South San Francisco,240
6,Santa Clara,201
7,Mountain View,200
8,Redwood City,186
9,Santa Clara Valley,168


We drop the last row which contains a None value. The location "San Francisco Bay Area" is not specific enough to assign the jobs to a city in the Bay Area, and Santa Clara Valley area contains 18 cities and towns so the jobs there cannot be assigned to specific cities either. Therefore, we drop these two rows as well. Finally, we convert the first column into a string and second column into numeric values.

In [26]:
df.dropna(inplace=True) #Drop the last column with a None value. 
df=df[~df.City.str.contains('San Francisco Bay Area|Santa Clara Valley')].reset_index(drop=True) #Drop rows which contain 'San Francisco Bay Area or 'Santa Clara Valley' in one of their cells.

df['City']=df['City'].astype(str) #Convert the first column into string
df['Job Openings']=df['Job Openings'].apply(pd.to_numeric) #Convert the second column in the dataframe to numeric values
df.dtypes

City            object
Job Openings     int64
dtype: object

### 2.2. Crime data in the cities with data scientist jobs: 

We can extract the violent and property crime rates per 1000 people for each city listed above. The crime rate data in all cities in California is available on the following Wikipedia page:

In [27]:
url='https://en.wikipedia.org/wiki/California_locations_by_crime_rate#Cities_and_other_agencies'

index = requests.get(url).text #Request the text data from the Wikipedia page
soup2 = BeautifulSoup(index, 'html.parser') 

We extract the table from the html file above which contains the violent and property crime rate data in all cities in California and put it into a dataframe:

In [28]:
mtable=soup2.find_all('table') #Find all tables in the document

#Extract all text in the table except the column names
df_crime=[
    [td.get_text(strip=True) for td in tr.find_all('td') if td.string or td.a]
    for tr in mtable[2].find_all('tr')[1:]
]

#Extract all column names in the table
Header=[]
for st in mtable[2].find_all('th'):
    st1=st.next_element.strip()
    if st.br:
        st2=st.find_next('br').next_element.strip()
    else: 
        st2=""
    Header.append(" ".join([st1, st2]))

#Create a dataframe which contains all the necessary data in the table
df_crime=pd.DataFrame(df_crime,columns=Header)

df_crime

,City/Agency,County,Population,Population density,Violent crimes,"Violent crime rate per 1,000 persons",Property crimes,"Property crime rate per 1,000 persons"
0,Adelanto,San Bernardino,"31,213",557.3,189,6.06,790,25.31
1,Agoura Hills,Los Angeles,"20,767","2,664.8",17,0.82,234,11.27
2,Alameda,Alameda,"77,048","7,378.7",145,1.88,"1,723",22.36
3,Albany,Alameda,"19,350","10,822.1",31,1.6,478,24.7
4,Alhambra,Los Angeles,"84,931","11,129.7",168,1.98,"1,743",20.52
5,Aliso Viejo,Orange,"50,671","7,323.5",35,0.69,273,5.39
6,Alturas,Modoc,"2,615","1,073.9",29,11.09,89,34.03
7,American Canyon,Napa,"20,379","3,351.3",55,2.7,568,27.87
8,Anaheim,Orange,"346,956","6,942.3","1,101",3.17,"8,196",23.62
9,Anderson,Shasta,"10,176","1,597.0",96,9.43,617,60.63


We inner join the two data sets with the job opening data by city and the crime rate data. We come up with the dataframe below which contains all cities in the Bay Area with job openings, and violent and property crime rates for these cities.

In [58]:
df_crime.rename(columns={'City/Agency ':'City'},inplace=True)


df_crime_rate= df_crime[['City','Violent crime rate per 1,000 persons','Property crime rate per 1,000 persons']]

df1 = df.merge(df_crime_rate, how='inner', on='City')

df1[['Violent crime rate per 1,000 persons','Property crime rate per 1,000 persons']]=df1[['Violent crime rate per 1,000 persons','Property crime rate per 1,000 persons']].apply(pd.to_numeric)

df1

,City,Job Openings,"Violent crime rate per 1,000 persons","Property crime rate per 1,000 persons"
0,San Francisco,1439,7.95,53.03
1,Sunnyvale,529,1.12,15.77
2,San Jose,258,3.21,24.34
3,Palo Alto,247,0.88,19.34
4,Menlo Park,241,1.56,16.96
5,South San Francisco,240,2.34,19.07
6,Santa Clara,201,1.34,26.98
7,Mountain View,200,1.98,20.42
8,Redwood City,186,2.37,21.11
9,Livermore,115,2.74,17.42


### 2.3. Venue and rating data for each city:

We can use the Foursquare API data to extract the nearby Turkish restaurants for each city and their ratings.

In [60]:
CLIENT_ID = 'KJ1V24Q1GYPAUPGCPKTQ130M0UXG21Q0S1W2SI2AGPHUP4WB' # my Foursquare ID
CLIENT_SECRET = 'IJLZNOPFSPLXD41DUOA4YIDZXCD3ZSAKMK4SXJTISANLZTS2' # my Foursquare Secret
VERSION = '20190321' # Foursquare API version

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: KJ1V24Q1GYPAUPGCPKTQ130M0UXG21Q0S1W2SI2AGPHUP4WB
CLIENT_SECRET:IJLZNOPFSPLXD41DUOA4YIDZXCD3ZSAKMK4SXJTISANLZTS2


#### Turkish Restaurants in San Francisco

We will first show in detail how we come up with the average rating of Turkish restaurants in San Francisco by using the Foursquare API. Then, we will apply the same logic to extract the average rating of Turkish restaurants in all cities listed in the final dataframe above.

We first search up to 20 Turkish restaurants near San Francisco, CA by using the category ID for "Turkish Restaurant" in [Foursquare venue categories list](https://developer.foursquare.com/docs/resources/categories). We set the limit as 20 because it is a premium request to get the details of a venue in Forsquare API, which includes the rating data. We have 13 cities with possibly 20 Turkish restaurants each. Given that we can only send 500 premium requests per day, the limit of 20 restaurants for each city seems to be a reasonable number considering the requests sent during debugging.

In [65]:
LIMIT=20
city='San Francisco, CA'
Turkish='4f04af1f2fb6e1c99f3db0bb'
# type your answer here
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&categoryId={}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, city, Turkish,VERSION, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=KJ1V24Q1GYPAUPGCPKTQ130M0UXG21Q0S1W2SI2AGPHUP4WB&client_secret=IJLZNOPFSPLXD41DUOA4YIDZXCD3ZSAKMK4SXJTISANLZTS2&near=San Francisco, CA&categoryId=4f04af1f2fb6e1c99f3db0bb&v=20190321&limit=20'

We send the get request for the above url and examine the data.

In [66]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c98049cdd579725dd239dcd'},
 'response': {'venues': [{'id': '4a6cc33df964a52091d11fe3',
    'name': 'A La Turca Restaurant',
    'location': {'address': '869 Geary St',
     'crossStreet': 'Larkin St.',
     'lat': 37.78614159981687,
     'lng': -122.41758355741563,
     'labeledLatLngs': [{'label': 'display',
       'lat': 37.78614159981687,
       'lng': -122.41758355741563}],
     'postalCode': '94109',
     'cc': 'US',
     'city': 'San Francisco',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['869 Geary St (Larkin St.)',
      'San Francisco, CA 94109',
      'United States']},
    'categories': [{'id': '4f04af1f2fb6e1c99f3db0bb',
      'name': 'Turkish Restaurant',
      'pluralName': 'Turkish Restaurants',
      'shortName': 'Turkish',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/turkish_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '305120',
     'url': 

In [68]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

We pull the id, name and category of each venue from the results and put the data into the dataframe below:

In [69]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['id','name','categories']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues

,id,name,categories
0,4a6cc33df964a52091d11fe3,A La Turca Restaurant,Turkish Restaurant
1,49fb9750f964a5205f6e1fe3,Eden's Mediterranean Turkish & Greek Restaurant,Mediterranean Restaurant
2,5a1baf41628c837b43898792,Doner Kebab,Doner Restaurant
3,59dfd5e0419a9e7fcc3637e1,Yaprak Doner,Turkish Restaurant
4,54f8caef498e72f9a012bd6a,Cobani Gyro & Kebab,Mediterranean Restaurant
5,51589757e4b0d20cd97f70ac,Hummus Mediterranean Kitchen,Mediterranean Restaurant
6,4a6dfb5df964a52082d31fe3,Turkish Kitchen,Turkish Restaurant
7,501602a7e4b053557197cb9a,Arkadaslarin yeri,Turkish Restaurant
8,51620309e4b09f3a6d85ac0b,Aspendos,Turkish Restaurant
9,4ff74283e4b045d12e3465dd,Turkish Kitchen Express,Turkish Restaurant


We then use the id of each venue above to get the details of each venue through Foursquare API. From these details, we extract the name and rating of each venue:

In [70]:
venue_ratings=pd.DataFrame(columns=['name','rating'])
for i in range(nearby_venues.shape[0]):
    
    VENUE_ID=nearby_venues.loc[i,'id']
    url1 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(VENUE_ID, CLIENT_ID, CLIENT_SECRET, VERSION)
    results1 = requests.get(url1).json()
    venue = results1['response']['venue']
    venue_details= json_normalize(venue) # flatten JSON
    filtered_columns = ['name','rating']
    venue_ratings = venue_ratings.append(venue_details.loc[:, filtered_columns])

venue_ratings



/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,name,rating
0,A La Turca Restaurant,8.0
0,Eden's Mediterranean Turkish & Greek Restaurant,NaN
0,Doner Kebab,NaN
0,Yaprak Doner,NaN
0,Cobani Gyro & Kebab,8.5
0,Hummus Mediterranean Kitchen,7.6
0,Turkish Kitchen,7.3
0,Arkadaslarin yeri,NaN
0,Aspendos,NaN
0,Turkish Kitchen Express,NaN


We drop the venues which are not rated and reset the index of the dataframe.

In [78]:
venue_ratings.dropna(inplace=True)
venue_ratings.reset_index(drop=True, inplace=True)
venue_ratings

,name,rating
0,A La Turca Restaurant,8.0
1,Cobani Gyro & Kebab,8.5
2,Hummus Mediterranean Kitchen,7.6
3,Turkish Kitchen,7.3
4,Troya,7.6
5,Kitchen Istanbul,8.2
6,SINI,6.2
7,Cheko's Cafe,7.9
8,Tuba - Authentic Turkish Restaurant,8.7


We finally calculate the average rating of the restaurants above:

In [74]:
venue_ratings.rating.mean()

7.777777777777778

#### The average rating of Turkish restaurants in 13 cities: 

We will now repeat the same procedure above to get the average rating of nearby restaurants in all cities specified in subsection 2.2.

In [79]:
#Loop through each city specified in the final dataframe in subsection 2.2
for j in range(df1.shape[0]):

    # Extract the restaurants in each city
    LIMIT=20
    city=df1.loc[j,'City']+', CA'
    query='Restaurant'
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&query={}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, city, query,VERSION, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    
    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['id','name','categories']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    venue_ratings1=pd.DataFrame(columns=['name','rating'])

    #Loop through each venue to pull the rating of each venue from the venue details
    for i in range(nearby_venues.shape[0]):
    
        VENUE_ID=nearby_venues.loc[i,'id']
        url1 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(VENUE_ID, CLIENT_ID, CLIENT_SECRET, VERSION)
        results1 = requests.get(url1).json()
        venue1 = results1['response']['venue']
        venue_details= json_normalize(venue1) # flatten JSON
        filtered_columns = ['name','rating']
        venue_ratings1 = venue_ratings1.append(venue_details.loc[:, filtered_columns])
        venue_ratings1.dropna(inplace=True)
        venue_ratings1.reset_index(drop=True)
    
    #Put the average rating of Turkish restaurants in each city into the 'Average Restaurant Ratings' column of the dataframe in subsection 2.2
    df1.loc[j,'Average Restaurant Ratings']=venue_ratings1.rating.mean()

df1

,City,Job Openings,"Violent crime rate per 1,000 persons","Property crime rate per 1,000 persons",Average Restaurant Ratings
0,San Francisco,1439,7.95,53.03,7.800000
1,Sunnyvale,529,1.12,15.77,6.890000
2,San Jose,258,3.21,24.34,7.092857
3,Palo Alto,247,0.88,19.34,7.294737
4,Menlo Park,241,1.56,16.96,7.321053
5,South San Francisco,240,2.34,19.07,7.247059
6,Santa Clara,201,1.34,26.98,6.968421
7,Mountain View,200,1.98,20.42,7.200000
8,Redwood City,186,2.37,21.11,7.194737
9,Livermore,115,2.74,17.42,7.123529


We will normalize the features above and calculate a "livability score" (a linear combination of the normalized values of all features above) for each city according to the criteria specified in the problem description and come up with a ranking of the cities according to their livability scores.